In [2]:
import pandas as pd
from datetime import datetime
import pickle

## Read in Rat Data - clean up & calculate results for last 4 quarters
* Number of inspections & number of positive inspections

In [3]:
df = pd.read_csv("Rodent_Inspection.csv")

In [4]:
df = df[df.INSPECTION_TYPE == 'INITIAL']
df = df[df.BOROUGH.isin(['Manhattan','Bronx'])]
df = df[df.HOUSE_NUMBER == df.HOUSE_NUMBER]
df['ADDRESS'] = df.apply(lambda x: str(x['HOUSE_NUMBER']).strip() + " " + str(x['STREET_NAME']).strip(), axis=1)

### Creating Quaterly Groupings

In [8]:
def convert_ym_2_yq(ym):
    l = ym.split()
    if int(l[1]) <=3 and int(l[1]) >=1:
        return l[0] + " Q1"
    if int(l[1]) <=6 and int(l[1]) >=4:
        return l[0] + " Q2"
    if int(l[1]) <=9 and int(l[1]) >=7:
        return l[0] + " Q3"
    if int(l[1]) <=12 and int(l[1]) >=10:
        return l[0] + " Q4"

In [9]:
df['INSPECTION_DATE_TS']= df.apply(lambda x: datetime.strptime(x['INSPECTION_DATE'], '%m/%d/%Y %H:%M:%S %p'), axis=1)
df['INSPECTION_MONTH'] = df.apply(lambda x: x['INSPECTION_DATE_TS'].month, axis=1)
df['INSPECTION_YEAR'] = df.apply(lambda x: x['INSPECTION_DATE_TS'].year, axis=1)
df['INSPECTION_YEAR_MONTH'] = df.apply(lambda x: str(x['INSPECTION_DATE_TS'].year) + " " + str(x['INSPECTION_DATE_TS'].month), axis=1)
df['INSPECTION_YEAR_Q'] = df.apply(lambda x: (convert_ym_2_yq(x['INSPECTION_YEAR_MONTH'])), axis=1)

### Keeping only pertinent columns
* Changing result to 1/0 to be more easily used for inspection calculations

In [10]:
col_to_keep = ['BLOCK','RESULT', 'INSPECTION_YEAR', 'INSPECTION_YEAR_Q' ]

df2 = df[col_to_keep].copy()
df2 = df2[df2['INSPECTION_YEAR'] > 2009]
df2 = df2[df2['INSPECTION_YEAR'] < 2018]
del df2['INSPECTION_YEAR']
df2['RESULT'] = df2.apply(lambda x: 1 if x['RESULT'] == 'Active Rat Signs' else 0, axis=1)
df2 = df2[df2.BLOCK > 0]

### Calculating total inspections & positive inspections
* It creates an annoying header format that needs to be cleaned up as well (thus the zip)

In [11]:
df3 = df2.groupby(['BLOCK','INSPECTION_YEAR_Q'], as_index=False).agg({'RESULT':['sum','count']})
a = df3.columns.get_level_values(0).tolist()
b = df3.columns.get_level_values(1).tolist()
df3.columns = [m+n for m,n in zip(a,b)]


### Grouping by Quarter & Block, then cartesian join to get all combinations
* Will need to do this to calculate quarters/blocks with no inspections

In [12]:
df_quarters = df2.groupby('INSPECTION_YEAR_Q', as_index=False)['RESULT'].sum()
del df_quarters['RESULT']

In [13]:
df09 = pd.DataFrame(['2018 Q1'], columns=list(['INSPECTION_YEAR_Q']))
df_quarters = df_quarters.append(df09)

In [14]:
df_blocks = df2.groupby('BLOCK', as_index=False)['RESULT'].sum()
del df_blocks['RESULT']

In [15]:
df_all = df_quarters.assign(foo=1).merge(df_blocks.assign(foo=1)).drop('foo', 1)

In [16]:
# always pickling for later
with open('df_all.pkl', 'wb') as picklefile:
    pickle.dump(df_all, picklefile)

### Create all combos, pickle for later
* Fill null cells w/ 0

In [17]:
df_combos = df_all.merge(df3, how='left', on=['INSPECTION_YEAR_Q','BLOCK']).copy()

In [18]:
df_combos.sort_values(['BLOCK','INSPECTION_YEAR_Q'], inplace=True)
df_combos.reset_index(inplace=True)
del df_combos['index']
df_combos.fillna(0, inplace=True)

In [19]:
#This get used separately in the model - save it!
with open('df_qr.pkl', 'wb') as picklefile:
    pickle.dump(df_combos, picklefile)

In [20]:
###########SPLIT#############

### Calculating last 4 quarter data for each block
* Pickle to be used in Calculating_Distances.ipynb

In [21]:
for i in range(1,5):
    df_combos['RESULT_S' + str(i)] = df_combos.groupby('BLOCK')['RESULTsum'].shift(i)
    df_combos['RESULT_C' + str(i)] = df_combos.groupby('BLOCK')['RESULTcount'].shift(i)
df_combos.dropna(inplace=True)

In [22]:
df_combos['last_4_ins'] = df_combos.apply(lambda x: x['RESULT_C1'] + x['RESULT_C2'] + 
                                                    x['RESULT_C3'] + x['RESULT_C4'] ,
                                          axis = 1)
df_combos['last_4_rats'] = df_combos.apply(lambda x: x['RESULT_S1'] + x['RESULT_S2'] + 
                                                    x['RESULT_S3'] + x['RESULT_S4'],
                                          axis = 1)
del df_combos['RESULT_C1']
del df_combos['RESULT_C2']
del df_combos['RESULT_C3']
del df_combos['RESULT_C4']
del df_combos['RESULT_S1']
del df_combos['RESULT_S2']
del df_combos['RESULT_S3']
del df_combos['RESULT_S4']

In [23]:
df_combos = df_combos.copy()
df_l4 = df_combos.iloc[:,0:6].copy()
del df_l4['RESULTsum']
del df_l4['RESULTcount']

with open('df_l4.pkl', 'wb') as picklefile:
    pickle.dump(df_l4, picklefile)